In [1]:
import numpy as np
import pandas as pd
from dLoader import DataLoader
from dLoader.utils import regroup

In [2]:
stocks_data = pd.read_json('Database/keeper.json', orient='index')
search_mask = stocks_data['Latest Price'].between(0, 10) & (stocks_data['Length in Database'] > (250 * 6))
tickers = stocks_data[search_mask].index.to_list()
print('There are {} tickers match the search.'.format(len(tickers)))

There are 253 tickers match the search.


In [3]:
np.random.seed(0)
ticker = np.random.choice(tickers)
print(ticker)

GPRO


In [23]:
data = DataLoader('AAPL').get_data('2018-01-01', '2018-12-31')